In [47]:
# import numpy as np
# import pandas as pd
# from pathlib import Path
# import os.path
# import matplotlib.pyplot as plt
# import tensorflow as tf

# # # Create a list with the filepaths for training and testing
# train_dir = Path('C:/Projects/Mobile robot in hazardous environment/Agricultural Robot/Tomato/Tomato_Images/Train')
# # train_filepaths = list(train_dir.glob(r'**/*.jpg'))

# test_dir = Path('C:/Projects/Mobile robot in hazardous environment/Agricultural Robot/Tomato/Tomato_Images/Test')
# # test_filepaths = list(test_dir.glob(r'**/*.jpg'))

# val_dir = Path('C:/Projects/Mobile robot in hazardous environment/Agricultural Robot/Tomato/Tomato_Images/Validation')
# # val_filepaths = list(test_dir.glob(r'**/*.jpg'))

# # def proc_img(filepath):

# #     labels = [str(filepath[i]).split("/")[-1] \
# #               for i in range(len(filepath))]

# #     filepath = pd.Series(filepath, name='Filepath').astype(str)
# #     labels = pd.Series(labels, name='Label')
    
# #     labels = pd.Series(labels, name='Label')
# #     # Concatenate filepaths and labels
# #     df = pd.concat([filepath, labels], axis=1)

# #     # Shuffle the DataFrame and reset index
# #     df = df.sample(frac=1).reset_index(drop = True)
    
# #     return df

# # train_df = os.listdir(train_filepaths)
# # test_df = proc_img(test_filepaths)
# # val_df = proc_img(val_filepaths)


In [49]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf

import splitfolders
splitfolders.ratio('./Tomato_Images', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 118 files [00:00, 174.04 files/s]


In [40]:
print('-- Training set --\n')
print(f'Number of pictures: {train_df.shape[0]}\n')
print(f'Number of different labels: {len(train_df.Label.unique())}\n')
print(f'Labels: {train_df.Label.unique()}')


-- Training set --

Number of pictures: 90

Number of different labels: 90

Labels: ['C:\\Projects\\Mobile robot in hazardous environment\\Agricultural Robot\\Tomato\\Tomato_Images\\Train\\Image_70.jpg'
 'C:\\Projects\\Mobile robot in hazardous environment\\Agricultural Robot\\Tomato\\Tomato_Images\\Train\\Image_88.jpg'
 'C:\\Projects\\Mobile robot in hazardous environment\\Agricultural Robot\\Tomato\\Tomato_Images\\Train\\Image_38.jpg'
 'C:\\Projects\\Mobile robot in hazardous environment\\Agricultural Robot\\Tomato\\Tomato_Images\\Train\\Image_90.jpg'
 'C:\\Projects\\Mobile robot in hazardous environment\\Agricultural Robot\\Tomato\\Tomato_Images\\Train\\Image_99.jpg'
 'C:\\Projects\\Mobile robot in hazardous environment\\Agricultural Robot\\Tomato\\Tomato_Images\\Train\\Image_82.jpg'
 'C:\\Projects\\Mobile robot in hazardous environment\\Agricultural Robot\\Tomato\\Tomato_Images\\Train\\Image_72.jpg'
 'C:\\Projects\\Mobile robot in hazardous environment\\Agricultural Robot\\Tomato\\

In [35]:
train_df.head(5)


,Filepath,Label
85,C:\Projects\Mobile robot in hazardous environm...,NaN
86,C:\Projects\Mobile robot in hazardous environm...,NaN
87,C:\Projects\Mobile robot in hazardous environm...,NaN
88,C:\Projects\Mobile robot in hazardous environm...,NaN
89,C:\Projects\Mobile robot in hazardous environm...,NaN


In [11]:
# # Create a DataFrame with one Label of each category
# df_unique = train_df.copy().drop_duplicates(subset=["Label"]).reset_index()

# # Display some pictures of the dataset
# fig, axes = plt.subplots(nrows=6, ncols=6, figsize=(8, 7),
#                         subplot_kw={'xticks': [], 'yticks': []})

# for i, ax in enumerate(axes.flat):
#     ax.imshow(plt.imread(df_unique.Filepath[i]))
#     ax.set_title(df_unique.Label[i], fontsize = 12)
# plt.tight_layout(pad=0.5)
# plt.show()


In [12]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_images = train_generator.flow_from_dataframe(
    dataframe=val_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 90 validated image filenames belonging to 90 classes.
Found 10 validated image filenames belonging to 10 classes.
Found 10 validated image filenames belonging to 10 classes.


In [13]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
pretrained_model.trainable = False


9412608/9406464 [==============================] - 26s 3us/step


In [19]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(90, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    batch_size = 10,
    epochs=5,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=2,
            restore_best_weights=True
        )
    ]
)

Epoch 1/5
3/3 [==============================] - ETA: 0s - loss: 4.6372 - accuracy: 0.0111   

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[10,90] labels_size=[10,10]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at \Temp\ipykernel_14064\1199187726.py:16) ]] [Op:__inference_test_function_31961]

Function call stack:
test_function
